In [8]:
from dotenv import load_dotenv
load_dotenv()

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from pprint import pprint
from dataclasses import dataclass
from typing import *

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

df = pd.read_csv('resources/bubbleflexe-rv.csv', usecols=[1, 2], names=['bsides', 'tt'])
df = df.drop(labels=0, axis=0).reset_index(drop=True)
for c in df.columns:
    df[c] = df[c].apply(lambda x: str(x).split(';'))
df['unified'] = df['bsides'] + df['tt']

In [10]:
df

,bsides,tt,unified
0,"[Some Love, Little Little, Body Talk, I Just, ...","[RBB, Umpah Umpah, Psycho, Monster, Naughty]","[Some Love, Little Little, Body Talk, I Just, ..."
1,"[Automatic, Huff N Puff, Oh Boy, Cool World, C...","[Happiness, Be Natural, Ice Cream Cake, Dumb D...","[Automatic, Huff N Puff, Oh Boy, Cool World, C..."
2,"[Oh Boy, Time Slip, Cool Hot Sweet Love, Rose ...","[Happiness, Ice Cream Cake, Dumb Dumb, Russian...","[Oh Boy, Time Slip, Cool Hot Sweet Love, Rose ..."
3,"[Candy, Body Talk, You Better Know, I Just, Ki...","[Ice Cream Cake, Dumb Dumb, Russian Roulette, ...","[Candy, Body Talk, You Better Know, I Just, Ki..."
4,"[Automatic, Day 1, Lucky Girl, Little Little, ...","[Ice Cream Cake, Dumb Dumb, Russian Roulette, ...","[Automatic, Day 1, Lucky Girl, Little Little, ..."
...,...,...,...
5697,"[Kingdom Come, So Good]",[Bad Boy],"[Kingdom Come, So Good, Bad Boy]"
5698,"[Sunny Side Up!, Eyes Locked Hands Locked, In ...","[Dumb Dumb, Russian Roulette, Red Flavor, Peek...","[Sunny Side Up!, Eyes Locked Hands Locked, In ..."
5699,"[Candy, Campfire, Cool Hot Sweet Love, Fool, Z...","[Russian Roulette, Peek-A-Boo, Cookie Jar, Mon...","[Candy, Campfire, Cool Hot Sweet Love, Fool, Z..."
5700,"[Automatic, Somethin Kinda Crazy, Candy, Huff ...","[Ice Cream Cake, Russian Roulette, Red Flavor,...","[Automatic, Somethin Kinda Crazy, Candy, Huff ..."


In [11]:
unique_bsides = set()
for i in df['bsides']:
    unique_bsides.update(i)

unique_tt = set()
for i in df['tt']:
    unique_tt.update(i)

unique_songs = unique_bsides.union(unique_tt)

In [12]:
a = spotify.artist('https://open.spotify.com/artist/1z4g3DjTBBZKhvAroFlhOM?si=UdOIkwEZRISZr0Zd0yWGyQ&dl_branch=1')
rv_id = a['id']

## Questions
1. For a given song, what is the frequency list of all other songs to it? 
2. How to use the size of favorite songs?
3. What about normalizing for song popularity

In [29]:
def get_flist(song: str) -> Dict:
    assert song in unique_songs
    flist = {}
    for s in unique_songs:
        flist[s] = 0
    mask = df['unified'].apply(lambda l: song in l)
    entries = df[mask]['unified']
    for u in entries:
        for s in u:
            flist[s] += 1
    
    return {k: v for k, v in sorted(flist.items(), key=lambda item: item[1])}

In [31]:
get_flist('My Dear')

{'nan': 0,
 'Stupid Cupid': 254,
 'Bad Dracula': 259,
 'Rose Scent Breeze': 259,
 "Don't U Wait No More": 263,
 'Attaboy': 271,
 'Campfire': 276,
 'Parade': 276,
 'First Time': 276,
 'Some Love': 278,
 'Huff N Puff': 281,
 "Lady's Room": 282,
 'Mosquito': 291,
 'Bing Bing': 291,
 'Time Slip': 293,
 "'Cause It's You": 293,
 'Mr. E': 294,
 'Last Love': 296,
 'Cool World': 302,
 'Uncover': 309,
 'Mojito': 311,
 'All Right': 312,
 'Diamond': 317,
 'Happiness': 319,
 'Red Dress': 321,
 'Aitai-tai': 322,
 "Jumpin'": 326,
 'Taste': 329,
 'Swimming Pool': 330,
 'Happily Ever After': 330,
 'Fool': 334,
 'Somethin Kinda Crazy': 334,
 'Carpool': 334,
 'Sappy': 337,
 'Feel Good': 337,
 'My Second Date': 337,
 'Take It Slow': 340,
 'With You': 340,
 'Be Natural': 342,
 'Rookie': 348,
 'Hit That Drum': 350,
 'Milkshake': 352,
 'Perfect 10': 353,
 'Sayonara': 354,
 'Cool Hot Sweet Love': 354,
 'Sunny Afternoon': 356,
 'Zoo': 359,
 'Power Up': 362,
 'Lucky Girl': 364,
 'About Love': 366,
 'Jelly': 368